Cross Model Testing: This file is my implementation of ehsan's code for  testing model A on the images generated by model B (FGSM)

In [ ]:
import sys
print(sys.executable)

In [ ]:
# FGSM-AlexNet - Running on full dataset: 3923 total images
# Put this into a Jupyter notebook cell
from __future__ import annotations
import os
from cornet import cornet_s
from pathlib import Path
from typing import List, Tuple, Dict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models

# -------------------------
# Dataset: ALL images = test set
# -------------------------
class AllImagesAsTestDataset(Dataset):
    """Wraps a list of (image_path, label) - used as the entire test set (no split)."""
    def __init__(self, samples: List[Tuple[str, int]], transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        p, label = self.samples[idx]
        img = Image.open(p).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

def build_all_test_samples(root: str, sort_filenames: bool = True) -> Tuple[List[Tuple[str,int]], Dict[int, str]]:
    """
    Walks `root` and builds a list of (image_path, label) for *all* files in each class folder.
    Label assigned by sorted folder order. Returns samples and class_map.
    """
    root_p = Path(root)
    assert root_p.exists(), f"Data root not found: {root}"
    class_dirs = sorted([d for d in root_p.iterdir() if d.is_dir()])
    samples = []
    class_map = {}
    for label, cls in enumerate(class_dirs):
        imgs = [p for p in cls.iterdir() if p.is_file()]
        if sort_filenames:
            imgs = sorted(imgs)
        for p in imgs:
            samples.append((str(p), label))
        class_map[label] = cls.name
    return samples, class_map

# -------------------------
# FGSM helper (normalized-space)
# -------------------------
def fgsm_perturb_from_grad(x: torch.Tensor, grad: torch.Tensor, epsilon: float) -> torch.Tensor:
    """Return perturbed inputs in normalized input space using gradient sign."""
    return torch.clamp(x + epsilon * grad.sign(), -10.0, 10.0).detach()

# -------------------------
# Evaluation (clean + FGSM)
# -------------------------
def evaluate_clean(model: nn.Module, loader: DataLoader, device: torch.device) -> float:
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return (correct / total) if total > 0 else 0.0

def evaluate_fgsm_cross(
        generating_model: nn.Module, 
        predicting_model: nn.Module,
        loader: DataLoader, 
        device: torch.device,
        epsilons: List[float], 
        loss_fn=None
) -> Dict[float, float]:
    if loss_fn is None:
        loss_fn = nn.CrossEntropyLoss()
    generating_model.eval()
    predicting_model.eval()
    results = {eps: {'correct': 0, 'total': 0} for eps in epsilons}

    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        # generating perturbed image using model A
        x.requires_grad = True
        out = generating_model(x)
        loss = loss_fn(out, y)

        generating_model.zero_grad()
        loss.backward()
        grad = x.grad.data
        # getting the perdiction using model B
        for eps in epsilons:
            x_adv = fgsm_perturb_from_grad(x, grad, eps)
            with torch.no_grad():
                out_adv = predicting_model(x_adv)
                preds = out_adv.argmax(dim=1)
                results[eps]['correct'] += (preds == y).sum().item()
                results[eps]['total'] += y.size(0)
        x.grad = None

    return {eps: (results[eps]['correct'] / results[eps]['total'] if results[eps]['total'] > 0 else 0.0)
            for eps in epsilons}

# -------------------------
# Pretrained model loaders
# -------------------------
def load_pretrained_alexnet(num_classes_expected: int, device: torch.device):
    model = models.alexnet(pretrained=True)
    if num_classes_expected != 1000:
        # replace final layer to match labels
        in_feats = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_feats, num_classes_expected)
        print(f"[warning] AlexNet: dataset has {num_classes_expected} classes != 1000; final layer replaced (random init).")
    return model.to(device)

def load_pretrained_vgg16(num_classes_expected: int, device: torch.device):
    model = models.vgg16(pretrained=True)
    if num_classes_expected != 1000:
        in_feats = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_feats, num_classes_expected)
        print(f"[warning] VGG16: dataset has {num_classes_expected} classes != 1000; final layer replaced (random init).")
    return model.to(device)

def load_pretrained_cornet(num_classes_expected: int, device: torch.device):
    # load CORnet-S (pretrained on imagenet)
    model = cornet_s(pretrained=True)

    # CORnet-S classifier is decoder
    if num_classes_expected != 1000:
        in_feats = model.decoder.in_features
        model.decoder = nn.Linear(in_feats, num_classes_expected)
        print(
            f"[warning] CORnet-S: dataset has {num_classes_expected} classes != 1000; "
            "final layer replaced (random init)."
        )
    return model.to(device)


In [3]:
def eval_pretrained_on_all_test(
    data_dir: str,
    gen_model_name: str,
    pred_model_name: str,
    batch_size: int = 128,
    num_workers: int = 4,
    device: str | None = None,
    epsilons: List[float] = (0.005, 0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5),
):
    """
    Evaluate pretrained AlexNet/VGG16 on ALL images in `data_dir` (root/class_x/*.jpg).
    All images are treated as test samples (no splitting, no training).
    Returns a dict with clean and adversarial accuracies per model.
    """
    device = torch.device(device if device else ("cuda" if torch.cuda.is_available() else "cpu"))
    print(f"[eval_pretrained_on_all_test] using device: {device}")

    samples, class_map = build_all_test_samples(data_dir)
    n_classes = len(class_map)
    print(f"Found {n_classes} classes and {len(samples)} total images (all used as test samples).")

    # Deterministic transform (224 x 224), same for all images
    input_size = 224
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ])

    test_ds = AllImagesAsTestDataset(samples, transform=transform)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    results = {}

    # loading generating models
    if gen_model_name == "alexnet":
        gen_model = load_pretrained_alexnet(n_classes, device)
    elif gen_model_name in ("cornet", "cornet_s"):
        gen_model = load_pretrained_cornet(n_classes, device)
    elif gen_model_name == "vgg16":
        gen_model = load_pretrained_vgg16(n_classes, device)
    else:
        print(f"[eval_pretrained_on_all_test] unknown model '{gen_model_name}' - skipping")

    # loading predictor models
    if pred_model_name == "alexnet":
        pred_model = load_pretrained_alexnet(n_classes, device)
    elif pred_model_name in ("cornet", "cornet_s"):
        pred_model = load_pretrained_cornet(n_classes, device)
    elif pred_model_name == "vgg16":
        pred_model = load_pretrained_vgg16(n_classes, device)
    else:
        print(f"[eval_pretrained_on_all_test] unknown model '{pred_model_name}' - skipping")

    gen_model.eval()
    pred_model.eval()
    
    clean_acc = evaluate_clean(gen_model, test_loader, device)
    print(f"{gen_model_name} clean accuracy: {clean_acc:.4f} = {clean_acc*100:.2f}%")

    clean_acc = evaluate_clean(pred_model, test_loader, device)
    print(f"{pred_model_name} clean accuracy: {clean_acc:.4f} = {clean_acc*100:.2f}%")

    cross_results = {}

    adv_acc = evaluate_fgsm_cross(
        generating_model=gen_model, 
        predicting_model=pred_model,
        loader=test_loader, 
        device=device,
        epsilons=list(epsilons), 
        loss_fn=nn.CrossEntropyLoss()
    )
    for eps, acc in adv_acc.items():
        print(f"{gen_model_name} generates & {pred_model_name} predicts | eps={eps:.4g} | acc={acc*100:.2f}%")
    
    cross_results[pred_model_name] = adv_acc

    results[gen_model_name] = {
        "clean_acc": clean_acc,
        "cross": cross_results,
    }

    print("[eval_pretrained_on_all_test] done.")
    return results

# -------------------------
# Example usage (edit path):
# -------------------------
# /imaging/mrahm326/val/


In [ ]:
# TEST: Alex generates --- COR predicts
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="alexnet",
    pred_model_name="cornet",
    batch_size=16, device="cuda:0", epsilons=[0.005]
    )
print(results)

[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


alexnet clean accuracy: 0.5631 = 56.31%
alexnet generates & cornet predicts | eps=0.005 | acc=71.50%
[eval_pretrained_on_all_test] done.
{'alexnet': {'clean_acc': 0.563089472342595, 'cross': {'cornet': {0.005: 0.7150140198827428}}}}


In [ ]:
# TEST: Alex generates --- COR predicts
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="alexnet",
    pred_model_name="cornet",
    batch_size=16, device="cuda:0", epsilons=[0.005, 0.01]
    )
print(results)

[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


alexnet clean accuracy: 0.5631 = 56.31%
cornet clean accuracy: 0.7214 = 72.14%
alexnet generates & cornet predicts | eps=0.005 | acc=71.50%
alexnet generates & cornet predicts | eps=0.01 | acc=70.76%
[eval_pretrained_on_all_test] done.
{'alexnet': {'clean_acc': 0.7213866938567423, 'cross': {'cornet': {0.005: 0.7150140198827428, 0.01: 0.7076217180729034}}}}


In [10]:
# Alex generates --- COR predicts
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="alexnet",
    pred_model_name="cornet",
    batch_size=16, device="cuda:0"
    )
print(results)

[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).
alexnet clean accuracy: 0.5631 = 56.31%
cornet clean accuracy: 0.7214 = 72.14%
alexnet generates & cornet predicts | eps=0.005 | acc=71.50%
alexnet generates & cornet predicts | eps=0.01 | acc=70.76%
alexnet generates & cornet predicts | eps=0.02 | acc=69.41%
alexnet generates & cornet predicts | eps=0.03 | acc=68.03%
alexnet generates & cornet predicts | eps=0.05 | acc=64.92%
alexnet generates & cornet predicts | eps=0.1 | acc=56.13%
alexnet generates & cornet predicts | eps=0.2 | acc=41.19%
alexnet generates & cornet predicts | eps=0.3 | acc=27.43%
alexnet generates & cornet predicts | eps=0.5 | acc=13.00%
[eval_pretrained_on_all_test] done.
{'alexnet': {'clean_acc': 0.7213866938567423, 'cross': {'cornet': {0.005: 0.7150140198827428, 0.01: 0.7076217180729034, 0.02: 0.6941116492480245, 0.03: 0.6803466734641855, 0.05: 0.6492480244710681, 0.1: 0.5613051236298751, 0.2: 

In [4]:
# Alex generates --- VGG16 predicts
print(f"ALEX generates --- VGG16 predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="alexnet",
    pred_model_name="vgg16",
    batch_size=16, device="cuda:0"
    )
print(results)

ALEX generates --- VGG16 predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use

alexnet clean accuracy: 0.5631 = 56.31%
vgg16 clean accuracy: 0.7043 = 70.43%
alexnet generates & vgg16 predicts | eps=0.005 | acc=69.77%
alexnet generates & vgg16 predicts | eps=0.01 | acc=69.26%
alexnet generates & vgg16 predicts | eps=0.02 | acc=67.52%
alexnet generates & vgg16 predicts | eps=0.03 | acc=65.69%
alexnet generates & vgg16 predicts | eps=0.05 | acc=61.99%
alexnet generates & vgg16 predicts | eps=0.1 | acc=52.10%
alexnet generates & vgg16 predicts | eps=0.2 | acc=31.63%
alexnet generates & vgg16 predicts | eps=0.3 | acc=17.77%
alexnet generates & vgg16 predicts | eps=0.5 | acc=5.68%
[eval_pretrained_on_all_test] done.
{'alexnet': {'clean_acc': 0.7043079276064237, 'cross': {'vgg16': {0.005: 0.6976803466734642, 0.01: 0.6925822074942646, 0.02: 0.675248534284986, 0.03: 0.6568952332398674, 0.05: 0.6199337241906704, 0.1: 0.5210298241141983, 0.2: 0.3163395360693347, 0.3: 0.17767015039510578, 0.5: 0.056844251848075456}}}}


In [5]:
# COR generates --- ALEX predicts
print(f"COR generates --- ALEX predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="cornet",
    pred_model_name="alexnet",
    batch_size=16, device="cuda:0"
    )
print(results)

COR generates --- ALEX predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).
cornet clean accuracy: 0.7214 = 72.14%
alexnet clean accuracy: 0.5631 = 56.31%
cornet generates & alexnet predicts | eps=0.005 | acc=55.80%
cornet generates & alexnet predicts | eps=0.01 | acc=55.47%
cornet generates & alexnet predicts | eps=0.02 | acc=54.42%
cornet generates & alexnet predicts | eps=0.03 | acc=53.50%
cornet generates & alexnet predicts | eps=0.05 | acc=49.99%
cornet generates & alexnet predicts | eps=0.1 | acc=42.37%
cornet generates & alexnet predicts | eps=0.2 | acc=27.53%
cornet generates & alexnet predicts | eps=0.3 | acc=15.01%
cornet generates & alexnet predicts | eps=0.5 | acc=4.03%
[eval_pretrained_on_all_test] done.
{'cornet': {'clean_acc': 0.563089472342595, 'cross': {'alexnet': {0.005: 0.5579913331633953, 0.01: 0.5546775426969156, 0.02: 0.5442263573795565, 0.03: 0.5350497068569972, 0.05: 0.49987254652052, 

In [ ]:
# COR generates --- VGG16 predicts
print(f"COR generates --- VGG16 predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="cornet",
    pred_model_name="vgg16",
    batch_size=16, device="cuda:0"
    )
print(results)

COR generates --- VGG16 predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).
cornet clean accuracy: 0.7214 = 72.14%
vgg16 clean accuracy: 0.7043 = 70.43%
cornet generates & vgg16 predicts | eps=0.005 | acc=68.03%
cornet generates & vgg16 predicts | eps=0.01 | acc=65.18%
cornet generates & vgg16 predicts | eps=0.02 | acc=60.31%
cornet generates & vgg16 predicts | eps=0.03 | acc=55.95%
cornet generates & vgg16 predicts | eps=0.05 | acc=48.38%
cornet generates & vgg16 predicts | eps=0.1 | acc=37.93%
cornet generates & vgg16 predicts | eps=0.2 | acc=24.37%
cornet generates & vgg16 predicts | eps=0.3 | acc=14.89%
cornet generates & vgg16 predicts | eps=0.5 | acc=5.17%
[eval_pretrained_on_all_test] done.
{'cornet': {'clean_acc': 0.7043079276064237, 'cross': {'vgg16': {0.005: 0.6803466734641855, 0.01: 0.6517970940606679, 0.02: 0.6031098648993117, 0.03: 0.5595207749171552, 0.05: 0.4838134081060413, 0.1: 0.37930155493

In [7]:
# VGG16 generates --- ALEX predicts
print(f"VGG16 generates --- ALEX predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="vgg16",
    pred_model_name="alexnet",
    batch_size=16, device="cuda:0"
    )
print(results)

VGG16 generates --- ALEX predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).
vgg16 clean accuracy: 0.7043 = 70.43%
alexnet clean accuracy: 0.5631 = 56.31%
vgg16 generates & alexnet predicts | eps=0.005 | acc=55.85%
vgg16 generates & alexnet predicts | eps=0.01 | acc=55.60%
vgg16 generates & alexnet predicts | eps=0.02 | acc=54.80%
vgg16 generates & alexnet predicts | eps=0.03 | acc=53.89%
vgg16 generates & alexnet predicts | eps=0.05 | acc=51.62%
vgg16 generates & alexnet predicts | eps=0.1 | acc=44.94%
vgg16 generates & alexnet predicts | eps=0.2 | acc=30.10%
vgg16 generates & alexnet predicts | eps=0.3 | acc=17.61%
vgg16 generates & alexnet predicts | eps=0.5 | acc=4.79%
[eval_pretrained_on_all_test] done.
{'vgg16': {'clean_acc': 0.563089472342595, 'cross': {'alexnet': {0.005: 0.5585011470813154, 0.01: 0.5559520774917155, 0.02: 0.5480499617639562, 0.03: 0.5388733112413969, 0.05: 0.5161865918939587, 0.1: 0.

In [ ]:
# VGG16 generates --- COR predicts
print(f"VGG16 generates --- COR predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="vgg16",
    pred_model_name="cornet",
    batch_size=16, device="cuda:0"
    )
print(results)

VGG16 generates --- COR predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).
vgg16 clean accuracy: 0.7043 = 70.43%
cornet clean accuracy: 0.7214 = 72.14%
vgg16 generates & cornet predicts | eps=0.005 | acc=70.02%
vgg16 generates & cornet predicts | eps=0.01 | acc=67.93%
vgg16 generates & cornet predicts | eps=0.02 | acc=64.11%
vgg16 generates & cornet predicts | eps=0.03 | acc=61.02%
vgg16 generates & cornet predicts | eps=0.05 | acc=55.72%
vgg16 generates & cornet predicts | eps=0.1 | acc=47.77%
vgg16 generates & cornet predicts | eps=0.2 | acc=37.80%
vgg16 generates & cornet predicts | eps=0.3 | acc=31.23%
vgg16 generates & cornet predicts | eps=0.5 | acc=19.27%
[eval_pretrained_on_all_test] done.
{'vgg16': {'clean_acc': 0.7213866938567423, 'cross': {'cornet': {0.005: 0.700229416263064, 0.01: 0.6793270456283457, 0.02: 0.6410910017843487, 0.03: 0.6102472597501912, 0.05: 0.5572266122865154, 0.1: 0.47769564109

In [11]:
# ALEX generates --- ALEX predicts
print(f"ALEX generates --- ALEX predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="alexnet",
    pred_model_name="alexnet",
    batch_size=16, device="cuda:0"
    )
print(results)

ALEX generates --- ALEX predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).
alexnet clean accuracy: 0.5631 = 56.31%
alexnet clean accuracy: 0.5631 = 56.31%
alexnet generates & alexnet predicts | eps=0.005 | acc=30.26%
alexnet generates & alexnet predicts | eps=0.01 | acc=16.37%
alexnet generates & alexnet predicts | eps=0.02 | acc=6.12%
alexnet generates & alexnet predicts | eps=0.03 | acc=3.03%
alexnet generates & alexnet predicts | eps=0.05 | acc=1.12%
alexnet generates & alexnet predicts | eps=0.1 | acc=0.54%
alexnet generates & alexnet predicts | eps=0.2 | acc=0.46%
alexnet generates & alexnet predicts | eps=0.3 | acc=0.28%
alexnet generates & alexnet predicts | eps=0.5 | acc=0.20%
[eval_pretrained_on_all_test] done.
{'alexnet': {'clean_acc': 0.563089472342595, 'cross': {'alexnet': {0.005: 0.3025745602854958, 0.01: 0.16365026765230692, 0.02: 0.0611776701503951, 0.03: 0.030333928116237575, 0.05: 0.0112159

In [12]:
# COR generates --- COR predicts
print(f"COR generates --- COR predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="cornet",
    pred_model_name="cornet",
    batch_size=16, device="cuda:0"
    )
print(results)

COR generates --- COR predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).
cornet clean accuracy: 0.7214 = 72.14%
cornet clean accuracy: 0.7214 = 72.14%
cornet generates & cornet predicts | eps=0.005 | acc=35.69%
cornet generates & cornet predicts | eps=0.01 | acc=19.60%
cornet generates & cornet predicts | eps=0.02 | acc=7.95%
cornet generates & cornet predicts | eps=0.03 | acc=5.10%
cornet generates & cornet predicts | eps=0.05 | acc=3.39%
cornet generates & cornet predicts | eps=0.1 | acc=2.98%
cornet generates & cornet predicts | eps=0.2 | acc=3.52%
cornet generates & cornet predicts | eps=0.3 | acc=3.93%
cornet generates & cornet predicts | eps=0.5 | acc=4.36%
[eval_pretrained_on_all_test] done.
{'cornet': {'clean_acc': 0.7213866938567423, 'cross': {'cornet': {0.005: 0.35686974254397147, 0.01: 0.19602345144022432, 0.02: 0.07953097119551364, 0.03: 0.05098139179199592, 0.05: 0.033902625541677285, 0.1: 0.02

In [4]:
# VGG generates --- VGG predicts
print(f"VGG generates --- VGG predicts\n\n")
results = eval_pretrained_on_all_test(
    "val/val/", gen_model_name="vgg16",
    pred_model_name="vgg16",
    batch_size=16, device="cuda:0"
    )
print(results)

VGG generates --- VGG predicts


[eval_pretrained_on_all_test] using device: cuda:0
Found 1000 classes and 3923 total images (all used as test samples).


/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nsakr2/.conda/envs/condavenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


vgg16 clean accuracy: 0.7043 = 70.43%
vgg16 clean accuracy: 0.7043 = 70.43%
vgg16 generates & vgg16 predicts | eps=0.005 | acc=27.96%
vgg16 generates & vgg16 predicts | eps=0.01 | acc=13.18%
vgg16 generates & vgg16 predicts | eps=0.02 | acc=5.51%
vgg16 generates & vgg16 predicts | eps=0.03 | acc=3.67%
vgg16 generates & vgg16 predicts | eps=0.05 | acc=2.96%
vgg16 generates & vgg16 predicts | eps=0.1 | acc=2.98%
vgg16 generates & vgg16 predicts | eps=0.2 | acc=3.62%
vgg16 generates & vgg16 predicts | eps=0.3 | acc=3.67%
vgg16 generates & vgg16 predicts | eps=0.5 | acc=2.75%
[eval_pretrained_on_all_test] done.
{'vgg16': {'clean_acc': 0.7043079276064237, 'cross': {'vgg16': {0.005: 0.2796329339790976, 0.01: 0.13178689778230945, 0.02: 0.055059903135355595, 0.03: 0.036706602090237066, 0.05: 0.029569207239357635, 0.1: 0.029824114198317615, 0.2: 0.036196788172317106, 0.3: 0.036706602090237066, 0.5: 0.027529951567677798}}}}
